In [1]:
import time
import numpy as np

In [2]:
import sc3nb as scn

# OSC communication
With the OSC communication module of sc3nb you can directly send and receive OSC packets.

Open Sound Control (OSC) is a networking protocol for sound and is used by SuperCollider to communicate between sclang and scsynth. sc3nb is itself a OSC client and server. This allows sc3nb to send and receive OSC traffic.

For more information on OSC and especially how Supercollider handles OSC packets please refer to the following links:

* [Open Sound Control Specification](http://opensoundcontrol.org/spec-1_0)
* [Server vs Client SuperCollider Guide](http://doc.sccode.org/Guides/ClientVsServer.html)
* [Server Command Reference](https://doc.sccode.org/Reference/Server-Command-Reference.html)
* [SuperCollider Synth Server Architecture](https://doc.sccode.org/Reference/Server-Architecture.html)

To see more information what messages are send and received set the logging level to INFO as seen below.

In [3]:
import logging
logging.basicConfig(level=logging.INFO)
# even more verbose logging is also avaible
# logging.basicConfig(level=logging.DEBUG)

In [4]:
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process...


INFO:sc3nb.process_handling:Popen args: ['C:\\Program Files\\SuperCollider-3.11.0\\sclang.exe']


Done.
Registering OSC /return callback in sclang...
Done.
Loading default SynthDescs
Done.
Booting SuperCollider Server...


INFO:sc3nb.process_handling:Popen args: ['C:\\Program Files\\SuperCollider-3.11.0\\scsynth.exe', '-u', '57110', '-l', '6', '-i', '2', '-o', '2', '-a', '1024', '-c', '4096', '-b', '1024', '-R', '0']


...

Done.


INFO:sc3nb.sc_objects.server:OSC msg received from ('127.0.0.1', 57121): ('/return', 57121)


INFO:sc3nb.sclang:Connecting <SCServer addr=('127.0.0.1', 57110), process=<Process 'scsynth' (running) pid=19104>> with <SCLang process=<Process 'sclang' (running) pid=18816>>


INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/return', b'SCgf\x00\x00\x00\x02\x00\x01\x17sc3nb_vol..


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/done', '/d_recv')


sc3nb serves as OSC server and as client of the SuperCollider server `scsynth`.<br>
You can also communicate with the SuperCollider interpreter `sclang` via OSC.<br>


You can see the current connection information with `sc.server.connection_info()`

In [5]:
(sc3nb_ip, sc3nb_port), receivers = sc.server.connection_info()

This instance is at ('127.0.0.1', 57130),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57121)
                 


In [6]:
(sc3nb_ip, sc3nb_port), receivers

(('127.0.0.1', 57130),
 {('127.0.0.1', 57110): 'scsynth', ('127.0.0.1', 57121): 'sclang'})

If you want to communicate via OSC with another receiver you could add it's name via `sc.server.add_receiver(name: str, ip: str, port: int)` or you can pass a custom receiver when sending OSC

In [7]:
sc.server.add_receiver("sc3nb", sc3nb_ip, sc3nb_port)

In [8]:
sc.server.connection_info()

This instance is at ('127.0.0.1', 57130),
Known receivers: "scsynth" at ('127.0.0.1', 57110)
                 "sclang" at ('127.0.0.1', 57121)
                 "sc3nb" at ('127.0.0.1', 57130)
                 


(('127.0.0.1', 57130),
 {('127.0.0.1', 57110): 'scsynth',
  ('127.0.0.1', 57121): 'sclang',
  ('127.0.0.1', 57130): 'sc3nb'})

## Sending OSC

You can send OSC with 

In [9]:
sc.server.send?

### Messages

Use the `OSCMessage` or the `python-osc` package to build a OscMessage

In [10]:
scn.OSCMessage?

In [11]:
msg = scn.OSCMessage("/s_new", ["s1", -1, 1, 1,])
sc.server.send(msg)

A shortcut for sending Messages is

In [12]:
sc.server.msg?

In [13]:
sc.server.msg("/s_new", ["s1", -1, 1, 1,])

a more complex example

In [14]:
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/d_removed', 'sc3nb_volumeAmpControl2')


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1, 0, 67108865, -56, 1, -32, -40)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 67108865, 0, 134217729, 1, 1, -1, -1)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 134217729, 0, 201326593, 67108865, 1, -1, -1)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 201326593, 0, 268435457, 134217729, 1, -1, -1..


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 268435457, 0, 335544321, 201326593, 1, -1, -1..


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 335544321, 0, 402653185, 268435457, 1, -1, -1..


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 402653185, 0, 469762049, 335544321, 1, -1, -1..


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 469762049, 0, -24, 402653185, 1, -1, -1)


**Note** that the timing is here under python's control, which is not very precise. Here comes the `Bundler` class into play.

**Remarks**:
* note that the python code returns immediately and all events remain in scsynth
* note that unfortunately scsynth has a limited buffer for OSC messages, so it is not viable to spawn thousends of events. scsynth will then simply reject OSC messages.
* this motivated (and is solved) with a TimedQueue, see below.

### Bundles

In [15]:
from sc3nb.osc.osc_communication import Bundler

To send one or multiple message(s) with a timetag as a OSC Bundle you should use the `Bundler` class

* Bundlers allow to specify a timetag and thus let scsynth control the timing, which is much better, if applicable.
* Bundler can be created by


In [16]:
Bundler?

To add messages to the Bundler use 

The prefered way of creating Bundlers for sending to the server is via

In [17]:
sc.server.bundler?

This will add the `sc.server.latency` time to the timetag. By default this is `0.0` but you can set it.

In [18]:
sc.server.latency

0.0

In [19]:
sc.server.latency = 0.1
sc.server.latency

0.1

A Bundler lets you add Messages with

In [20]:
Bundler.add?

In [21]:
msg1 = scn.OSCMessage("/s_new", ["s2", 1001, 1, 1,])
msg2 = scn.OSCMessage("/n_free", [1001])
sc.server.bundler().add(msg1).add(0.4, msg2).send()

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459AC1E48> contents size 2 


Simpler is the usage of the context manager:


This means you can use `with` for better handling 

In [22]:
with sc.server.bundler() as bundler:
    bundler.add(0.0, msg1)
    bundler.add(0.3, msg2)

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A97208> contents size 2 


Here are some different styles of writing the same Sound with the Bundler features 

 - server Bundler with add

In [23]:
with sc.server.bundler(send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", 1001, 1, 1,])
    bundler.add(0.3, "/n_free", [1001])

In [24]:
dg1 = bundler.build(time_offset=0).dgram  # we set the time_offset explicitly so all Bundle datagrams are the same
dg1

AttributeError: 'Bundler' object has no attribute 'build'

 - Bundler with explict latency set and using add

In [25]:
with Bundler(sc.server.latency, send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", 1001, 1, 1])
    bundler.add(0.3, "/n_free", [1001])

In [26]:
dg2 = bundler.build(time_offset=0).dgram
dg2

AttributeError: 'Bundler' object has no attribute 'build'

 - server Bundler with implicit latency and using bundled messages

In [27]:
with sc.server.bundler(send_on_exit=False) as bundler:
    sc.server.msg("/s_new", ["s2", 1001, 1, 1,], bundle=True)
    bundler.wait(0.3)
    sc.server.msg("/n_free", [1001], bundle=True)

In [28]:
dg3 = bundler.build(time_offset=0).dgram
dg3

AttributeError: 'Bundler' object has no attribute 'build'

In [29]:
assert dg1 == dg2 and dg1 == dg2, "The datagrams are not the same"

NameError: name 'dg1' is not defined

Note that you can use the Bundler with the Synth and Group for easier Message creation.

Also make sure to look at the [Automatic Bundling Feature](#Automatic-Bundling) which is using the bundled messages (`msg(..., bundle=True`)

In [30]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(10)):
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        bundler.add(onset, scn.Synth("s1", {"freq": freq, "dur": 1.5, "num": abs(r)+1}, new=False).new(return_msg=True))

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/return', b'#bundle\x00\xe4kRy\rX\x1c \x00\x00\x00 \x..


INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459ACFC88> contents size 10 


#### Bundler Timestamp

small numbers are times in seconds relative to `time.time()` evaluated at the time of sending

In [31]:
sc.server.bundler(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459ACFCC8> contents size 1 


INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459ACFC48> contents size 1 


**Attention:** 

Sending bundles with relative times could lead to unprecise timings.
If you care about precise timings you should

* use one bundler with multiple messages (if you care about the timings relative to each other) or
* provide a explict timetag (>1e6) to specify absolute times (see next examples)

One Bundler with multiple messages

In [32]:
bundler = sc.server.bundler()
bundler.add(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1])
bundler.add(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1])
bundler.send()  # a tone starts in 1.0s

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A93C48> contents size 2 


using `time.time()+timeoffset` for absolute times

In [33]:
t0 = time.time()
sc.server.bundler(t0 + 0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(t0 + 1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A93CC8> contents size 1 


INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A93CC8> contents size 1 


In [34]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(100)):
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        msg_params = ["s1", -1, 1, 0, "freq", freq, "dur", 1.5, "num", abs(r)+1]
        bundler.add(onset, "/s_new", msg_params)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1001, 1, -1, -1, 0)


INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A97B88> contents size 100 


#### Nesting Bundlers

You can even nest bundler. This will recalculate the time relative to the sending time of the outermost bundler

In [35]:
with sc.server.bundler() as bundler_outer:
    with sc.server.bundler() as bundler:
        sc.server.msg("/s_new", ["s2", 1001, 1, 1,], bundle=True)
        bundler.wait(0.3)
        sc.server.msg("/n_free", [1001], bundle=True)
    bundler_outer.wait(0.4)
    bundler_outer.add(bundler)

INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A93C08> contents size 2 


### Automatic Bundling

Probably the most convenient way of sending OSC is by using the Automatic Bundling Feature.

This allows you to simply use the SuperCollider Objects Synth and Group in the Context Manager of a Bundler and they will be automatically captured and stored. 

In [36]:
with sc.server.bundler() as bundler:
    synth = scn.Synth("s2")
    bundler.wait(0.3)
    synth.set("freq", 1000)
    bundler.wait(0.1)
    synth.free()
synth.wait()

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/fail', '/s_new', 'duplicate node ID')


INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/return', b'#bundle\x00\xe4kRy\rX\x1c \x00\x00\x00 \x..


INFO:sc3nb.osc.osc_communication:send to scsynth : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459A78E88> contents size 3 


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20011, 67108865, -1, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/fail', '/s_new', 'duplicate node ID')


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 1001, 1, -1, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/fail', '/n_free', 'Node 1001 not found')


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/fail', '/n_free', 'Node 1001 not found')


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 20011, 67108865, -1, -1, 0)


**Note** that it is important that we only wait on the Synth after the context of the Bundler is closed.

If you would call `synth.wait()` in the Bundler context it would wait before sending the /s_new Message to the server and then wait forever (or until timeout) for the /n_end notification.

In [37]:
try:
    with sc.server.bundler() as bundler:
        synth = scn.Synth("s2")
        bundler.wait(0.3)
        synth.set("freq", 1000)
        bundler.wait(0.1)
        synth.free()
        synth.wait(timeout=2)  # without a timeout this would hang forever
except RuntimeError as error:
    print(error)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1001, 1, -1, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_end', 1001, 1, -1, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20003, 67108865, -1, -1, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20005, 67108865, -1, 20003, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20010, 67108865, -1, 20005, 0)


Aborting. Exception raised in bundler: TimeoutError Timed out waiting for synth.


## Receiving OSC

sc3nb is receiving OSC messages with the help of queues, for each OSC address where we want to receive messages there is a AddressQueue

In [38]:
sc.server.msg_queues

{<MasterControlReply.STATUS_REPLY: '/status.reply'>: AddressQueue /status.reply : [],
 <MasterControlReply.SYNCED: '/synced'>: AddressQueue /synced : [],
 <MasterControlReply.VERSION_REPLY: '/version.reply'>: AddressQueue /version.reply : [],
 <NodeCommand.SET: '/n_set'>: AddressQueue /n_set : [],
 <NodeCommand.SETN: '/n_setn'>: AddressQueue /n_setn : [],
 <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>: AddressQueue /g_queryTree.reply : [],
 <NodeReply.INFO: '/n_info'>: AddressQueue /n_info : [],
 <BufferReply.INFO: '/b_info'>: AddressQueue /b_info : [],
 <BufferCommand.SET: '/b_set'>: AddressQueue /b_set : [],
 <BufferCommand.SETN: '/b_setn'>: AddressQueue /b_setn : [],
 <ControlBusCommand.SET: '/c_set'>: AddressQueue /c_set : [],
 <ControlBusCommand.SETN: '/c_setn'>: AddressQueue /c_setn : [],
 <ReplyAddress.RETURN_ADDR: '/return'>: AddressQueue /return : [],
 '/done/quit': AddressQueue /quit : [],
 '/done/notify': AddressQueue /notify : [],
 '/done/d_recv': AddressQueue /d_recv

### Getting replies

For some outgoing Messages there is a incoming Message defined.

This means sending such a message will automatically wait for the incoming Message at the corresponding Queue and return the result.

An example for this is `/sync {sync_id} -> /synced {sync_id}` 

In [39]:
sc.server.msg("/sync", 12345)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 12345)


12345

See all `(outgoing message, incoming message)` pairs:

In [40]:
sc.server.reply_addresses

{<MasterControlCommand.STATUS: '/status'>: <MasterControlReply.STATUS_REPLY: '/status.reply'>,
 <MasterControlCommand.SYNC: '/sync'>: <MasterControlReply.SYNCED: '/synced'>,
 <MasterControlCommand.VERSION: '/version'>: <MasterControlReply.VERSION_REPLY: '/version.reply'>,
 <SynthCommand.S_GET: '/s_get'>: <NodeCommand.SET: '/n_set'>,
 <SynthCommand.S_GETN: '/s_getn'>: <NodeCommand.SETN: '/n_setn'>,
 <GroupCommand.QUERY_TREE: '/g_queryTree'>: <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>,
 <NodeCommand.QUERY: '/n_query'>: <NodeReply.INFO: '/n_info'>,
 <BufferCommand.QUERY: '/b_query'>: <BufferReply.INFO: '/b_info'>,
 <BufferCommand.GET: '/b_get'>: <BufferCommand.SET: '/b_set'>,
 <BufferCommand.GETN: '/b_getn'>: <BufferCommand.SETN: '/b_setn'>,
 <ControlBusCommand.GET: '/c_get'>: <ControlBusCommand.SET: '/c_set'>,
 <ControlBusCommand.GETN: '/c_getn'>: <ControlBusCommand.SETN: '/c_setn'>,
 <MasterControlCommand.QUIT: '/quit'>: '/done/quit',
 <MasterControlCommand.NOTIFY: '/notify'>: 

You can get the reply address via

In [41]:
sc.server.get_reply_address("/sync")

<MasterControlReply.SYNCED: '/synced'>

or

In [42]:
sc.server.reply_addresses["/sync"]

<MasterControlReply.SYNCED: '/synced'>

If we specify `await_reply=False` the message will be kept in the queue

In [43]:
sc.server.msg("/sync", 1, await_reply=False)

In [44]:
sc.server.msg_queues[sc.server.get_reply_address("/sync")]

AddressQueue /synced : []

In [45]:
sc.server.msg("/sync", 2, await_reply=False)

In [46]:
sc.server.msg_queues[sc.server.reply_addresses["/sync"]]

AddressQueue /synced : []

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 1)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 2)


In [47]:
sc.server.msg_queues["/synced"]

AddressQueue /synced : [1, 2]

You can see how many values were hold.

In [48]:
sc.server.msg_queues["/synced"].skips

2

Notice that these hold messages will be skipped. 

In [49]:
sc.server.msg("/sync", 3, await_reply=True)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 3)


3

In [50]:
sc.server.msg_queues["/synced"]

AddressQueue /synced : []

Therefore you should retrieve them with `get` and set `skip=False` if you care for old values in the queue and dont want them to be skipped.

In [51]:
sc.server.msg("/sync", 42, await_reply=False)
sc.server.msg_queues["/synced"].get(skip=False)

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 20004, 67108865, -1, 20010, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/synced', 42)


42

### Custom Message Queues

If you want to get additional OSC Messages you need to create a custom MessageQueue

In [52]:
from sc3nb.osc.osc_communication import MessageQueue

In [53]:
MessageQueue?

In [54]:
mq = MessageQueue("/test")

In [55]:
sc.server.add_msg_queue(mq)

In [56]:
sc.server.msg("/test", ["Hi!"], receiver="sc3nb")

INFO:sc3nb.sc_objects.server:OSC msg received from sc3nb: ('/test', 'Hi!')


In [57]:
sc.server.msg_queues["/test"]

AddressQueue /test : ['Hi!']

In [58]:
sc.server.msg("/test", ["Hello!"], receiver="sc3nb")

INFO:sc3nb.sc_objects.server:OSC msg received from sc3nb: ('/test', 'Hello!')


In [59]:
sc.server.msg_queues["/test"]

AddressQueue /test : ['Hi!', 'Hello!']

In [60]:
sc.server.msg_queues["/test"].get()

'Hi!'

In [61]:
sc.server.msg_queues["/test"].get()

'Hello!'

If you want to create a pair of a outgoing message that will receive a certain incomming message you need to specify it via the `out_addr` arugment of `add_msg_queue` or you could use the shortcut for this `add_msg_pairs`

In [62]:
sc.server.add_msg_pairs?

In [63]:
sc.server.add_msg_pairs({"/hi": "/hi.reply"})

Lets use `OSCdef` in sclang to send us replies.

In [64]:
%%scv
OSCdef.newMatching("say_hi", {|msg, time, addr, recvPort| addr.sendMsg("/hi.reply", "Hello there!")}, '/hi');

-> OSCdef(say_hi, /hi, nil, nil, nil)


In [65]:
sc.server.msg("/hi", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/hi.reply', 'Hello there!')


'Hello there!'

There is also the class MessageQueueCollection, this allows to create multiple MessageQueues for a multiple `addess/subaddresses` combination

In [66]:
from sc3nb.osc.osc_communication import MessageQueueCollection

In [67]:
MessageQueueCollection?

In [68]:
mqc = MessageQueueCollection("/collect", ["/address1", "/address2"])
sc.server.add_msg_queue_collection(mqc)

In [69]:
mqc = MessageQueueCollection("/auto_collect")
sc.server.add_msg_queue_collection(mqc)

In [70]:
sc.server.reply_addresses

{<MasterControlCommand.STATUS: '/status'>: <MasterControlReply.STATUS_REPLY: '/status.reply'>,
 <MasterControlCommand.SYNC: '/sync'>: <MasterControlReply.SYNCED: '/synced'>,
 <MasterControlCommand.VERSION: '/version'>: <MasterControlReply.VERSION_REPLY: '/version.reply'>,
 <SynthCommand.S_GET: '/s_get'>: <NodeCommand.SET: '/n_set'>,
 <SynthCommand.S_GETN: '/s_getn'>: <NodeCommand.SETN: '/n_setn'>,
 <GroupCommand.QUERY_TREE: '/g_queryTree'>: <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>,
 <NodeCommand.QUERY: '/n_query'>: <NodeReply.INFO: '/n_info'>,
 <BufferCommand.QUERY: '/b_query'>: <BufferReply.INFO: '/b_info'>,
 <BufferCommand.GET: '/b_get'>: <BufferCommand.SET: '/b_set'>,
 <BufferCommand.GETN: '/b_getn'>: <BufferCommand.SETN: '/b_setn'>,
 <ControlBusCommand.GET: '/c_get'>: <ControlBusCommand.SET: '/c_set'>,
 <ControlBusCommand.GETN: '/c_getn'>: <ControlBusCommand.SETN: '/c_setn'>,
 <MasterControlCommand.QUIT: '/quit'>: '/done/quit',
 <MasterControlCommand.NOTIFY: '/notify'>: 

In [71]:
sc.server.msg_queues

{<MasterControlReply.STATUS_REPLY: '/status.reply'>: AddressQueue /status.reply : [],
 <MasterControlReply.SYNCED: '/synced'>: AddressQueue /synced : [],
 <MasterControlReply.VERSION_REPLY: '/version.reply'>: AddressQueue /version.reply : [],
 <NodeCommand.SET: '/n_set'>: AddressQueue /n_set : [],
 <NodeCommand.SETN: '/n_setn'>: AddressQueue /n_setn : [],
 <GroupReply.QUERY_TREE_REPLY: '/g_queryTree.reply'>: AddressQueue /g_queryTree.reply : [],
 <NodeReply.INFO: '/n_info'>: AddressQueue /n_info : [],
 <BufferReply.INFO: '/b_info'>: AddressQueue /b_info : [],
 <BufferCommand.SET: '/b_set'>: AddressQueue /b_set : [],
 <BufferCommand.SETN: '/b_setn'>: AddressQueue /b_setn : [],
 <ControlBusCommand.SET: '/c_set'>: AddressQueue /c_set : [],
 <ControlBusCommand.SETN: '/c_setn'>: AddressQueue /c_setn : [],
 <ReplyAddress.RETURN_ADDR: '/return'>: AddressQueue /return : [],
 '/done/quit': AddressQueue /quit : [],
 '/done/notify': AddressQueue /notify : [],
 '/done/d_recv': AddressQueue /d_recv

In [72]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address1', "toast".scramble)}, '/address1');

-> OSCdef(ab, /address1, nil, nil, nil)


In [73]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address2', "sonification".scramble)}, '/address2');

-> OSCdef(ab, /address2, nil, nil, nil)


In [74]:
sc.server.msg("/address1", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/collect', '/address1', 'tatos')


'tatos'

In [75]:
sc.server.msg("/address2", receiver="sclang")

INFO:sc3nb.sc_objects.server:OSC msg received from sclang: ('/collect', '/address2', 'innsicfaotoi')


'innsicfaotoi'

## Examples 

### Creating an OSC responder and msg to sclang for synthesis

In [76]:
%%scv
OSCdef(\dinger, { | msg, time, addr, recvPort |
    var freq = msg[2];
    {Pulse.ar(freq, 0.04, 0.3)!2 * EnvGen.ar(Env.perc, doneAction:2)}.play()
}, '/ding')

-> OSCdef(dinger, /ding, nil, nil, nil)


In [77]:
with scn.Bundler(receiver=sc.lang.addr):
    for i in range(5):
        sc.server.msg("/ding", ["freq", 1000-5*i], bundle=True)

INFO:sc3nb.osc.osc_communication:send to sclang : <sc3nb.osc.osc_communication.Bundler object at 0x000001F459AFD4C8> contents size 5 


In [78]:
for i in range(5):
    sc.server.msg("/ding", ["freq", 1000-5*i], receiver=sc.lang.addr)

In [79]:
%scv OSCdef.freeAll()

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1001, 1, -1, 1000, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1002, 1, -1, 1001, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1003, 1, -1, 1002, 0)


-> OSCdef


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1004, 1, -1, 1003, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1005, 1, -1, 1004, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1006, 1, -1, 1005, 0)


INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/n_go', 1007, 1, -1, 1006, 0)


In [80]:
sc.exit()

INFO:sc3nb.sc_objects.server:OSC msg received from scsynth: ('/done', '/quit')
